# PROBLEMA 1

In [71]:
using Random, Distributions

# Parámetros
μ = 2.0
σ2 = 1.5
n = 10


10

### a) Simulación de observaciones

Dada una variable aleatoria $ Y_i \sim \text{Normal}(\mu, \sigma^2) $, generamos una muestra de tamaño $ n = 10 $. Cada observación se obtiene de la siguiente manera:

$$
y_i \sim \text{Normal}(\mu, \sigma^2), \quad i = 1, 2, \dots, 10
$$


In [72]:
# a) Generar observaciones
y = rand(Normal(μ, sqrt(σ2)), n)
println("Observaciones simuladas: ", y)

Observaciones simuladas: [1.098645090828223, -0.1472945661191872, 2.2068525185559404, 2.7879610634525993, 3.926574233118528, 3.807456216781026, 1.9699011016821755, 1.8887591696650845, 0.9172911046082523, 2.0136345733098664]


### b) Función de verosimilitud y log-verosimilitud

La **función de verosimilitud** para un conjunto de observaciones $ y = (y_1, y_2, \dots, y_n) $ bajo la suposición de que los $ Y_i $ son independientes y distribuidos como $ \text{Normal}(\mu, \sigma^2) $ está dada por:

$$
L(\mu, \sigma^2 \mid y) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(y_i - \mu)^2}{2\sigma^2}\right)
$$

La **log-verosimilitud** es la siguiente:

$$
\ell(\mu, \sigma^2 \mid y) = -\frac{n}{2} \log(2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - \mu)^2
$$


In [73]:
# b) Función de verosimilitud
function verosimilitud(y, μ, σ2)
    n = length(y)
    (1 / ((2 * π * σ2)^(n/2))) * exp(-sum((y .- μ).^2) / (2 * σ2))
end

# Log-verosimilitud
function log_verosimilitud(y, μ, σ2)
    n = length(y)
    -(n/2) * log(2 * π * σ2) - sum((y .- μ).^2) / (2 * σ2)
end

println("Verosimilitud: ", verosimilitud(y, μ, σ2))
println("Log-verosimilitud: ", log_verosimilitud(y, μ, σ2))


Verosimilitud: 1.1629415861167337e-7
Log-verosimilitud: -15.967143005581974


### c) Función score y la información de Fisher

La **función score** es la derivada de la log-verosimilitud con respecto a los parámetros $ \mu $ y $ \sigma^2 $:

Para $ \mu $:

$$
S_{\mu}(\mu, \sigma^2) = \frac{\partial \ell(\mu, \sigma^2 \mid y)}{\partial \mu} = \frac{1}{\sigma^2} \sum_{i=1}^{n} (y_i - \mu)
$$

Para $ \sigma^2 $:

$$
S_{\sigma^2}(\mu, \sigma^2) = \frac{\partial \ell(\mu, \sigma^2 \mid y)}{\partial \sigma^2} = -\frac{n}{2\sigma^2} + \frac{1}{2\sigma^4} \sum_{i=1}^{n} (y_i - \mu)^2
$$

La **información de Fisher** es la matriz de varianza-covarianza inversa de los estimadores de máxima verosimilitud, cuyas entradas son los valores 
esperados de los productos de las derivadas de la log-verosimilitud:

Para $ \mu $ y $ \sigma^2 $:

$$
\mathcal{I}(\mu, \sigma^2) = 
\begin{pmatrix}
\frac{n}{\sigma^2} & 0 \\
0 & \frac{n}{2\sigma^4}
\end{pmatrix}
$$

In [74]:
# c) Función score para μ y σ2
function score(y, μ, σ2)
    n = length(y)
    dL_dμ = sum(y .- μ) / σ2
    dL_dσ2 = -(n / (2 * σ2)) + sum((y .- μ).^2) / (2 * σ2^2)
    return dL_dμ, dL_dσ2
end

# Información de Fisher
function fisher_information(y, μ, σ2)
    n = length(y)
    I_μμ = n / σ2
    I_μσ2 = 0
    I_σ2σ2 = n / (2 * σ2^2)
    return I_μμ, I_μσ2, I_σ2σ2
end

println("Score: ", score(y, μ, σ2))
println("Información de Fisher: ", fisher_information(y, μ, σ2))


Score: (0.3131870039216724, -0.16637857800371814)
Información de Fisher: (6.666666666666667, 0, 2.2222222222222223)


### d) Intervalos de confianza usando el estimador de máxima verosimilitud

Los **estimadores de máxima verosimilitud** para $ \mu $ y $ \sigma^2 $ son:

$$
\hat{\mu} = \frac{1}{n} \sum_{i=1}^{n} y_i, \quad \hat{\sigma}^2 = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{\mu})^2
$$

Para el intervalo de confianza de $ \mu $:

$$
\hat{\mu} \pm z_{\alpha/2} \cdot \sqrt{\frac{\hat{\sigma}^2}{n}}
$$

donde $ z_{\alpha/2} $ es el valor crítico de la distribución normal estándar.

Para $ \sigma^2 $, el intervalo de confianza es:

$$
\left( \frac{(n-1)\hat{\sigma}^2}{\chi^2_{\alpha/2, n-1}}, \frac{(n-1)\hat{\sigma}^2}{\chi^2_{1-\alpha/2, n-1}} \right)
$$

donde $ \chi^2_{\alpha, k} $ es el valor crítico de la distribución chi-cuadrado con $ k $ grados de libertad.

In [75]:
# d) Estimador de máxima verosimilitud para μ y σ2
μ_hat = mean(y)
σ2_hat = var(y)

# Intervalos de confianza
α = 0.05  # Nivel de significancia
z = quantile(Normal(0, 1), 1 - α/2)

conf_μ = (μ_hat - z * sqrt(σ2_hat / n), μ_hat + z * sqrt(σ2_hat / n))
conf_σ2 = (σ2_hat * (n - 1) / quantile(Chisq(n - 1), 1 - α/2), σ2_hat * (n - 1) / quantile(Chisq(n - 1), α/2))

println("Intervalo de confianza para μ: ", conf_μ)
println("Intervalo de confianza para σ2: ", conf_σ2)



Intervalo de confianza para μ: (1.26765505612418, 2.8263010450523223)
Intervalo de confianza para σ2: (0.748010340936224, 5.269323935201218)


### e) Intervalos de confianza usando la devianza

La **devianza** es una medida de la calidad del ajuste del modelo. Se define como:

$$
D = -2 \cdot \ell(\hat{\mu}, \hat{\sigma}^2 \mid y)
$$

Para construir intervalos de confianza usando la devianza, se compara con la distribución $ \chi^2 $ asintótica bajo ciertas condiciones del modelo.

In [76]:
# e) Función de devianza
function devianza(y, μ_hat, σ2_hat)
    -2 * log_verosimilitud(y, μ_hat, σ2_hat)
end

# Intervalos de confianza con la devianza (asume chi-cuadrado)
conf_devianza = [devianza(y, μ_hat, σ2_hat) + quantile(Chisq(1), α/2), devianza(y, μ_hat, σ2_hat) + quantile(Chisq(1), 1 - α/2)]
println("Intervalo de confianza usando la devianza: ", conf_devianza)


Intervalo de confianza usando la devianza: [31.960487866261573, 36.98339198445929]


### f) Prueba de hipótesis usando la razón de verosimilitudes

Para probar la hipótesis $ H_0: \mu_1 = \mu_2 $ contra $ H_1: \mu_1 \neq \mu_2 $, usamos la **razón de verosimilitudes**:

$$
\Lambda = -2\left[ \ell(\hat{\mu}, \hat{\sigma}^2 \mid y) - (\ell(\hat{\mu}_1, \hat{\sigma}^2_1 \mid y_1) + \ell(\hat{\mu}_2, \hat{\sigma}^2_2 \mid y_2))\right]
$$

Este estadístico bajo $ H_0 $ se distribuye asintóticamente como una $ \chi^2 $ con un grado de libertad, y el p-valor asociado nos permite tomar una decisión sobre $ H_0 $.


In [77]:
# f) Simular segunda muestra
x = rand(Normal(3.0, sqrt(σ2)), n)

# Hipótesis nula: μ1 = μ2
# Hipótesis alternativa: μ1 ≠ μ2
logL0 = log_verosimilitud([y; x], mean([y; x]), σ2_hat)  # Bajo H0
logL1 = log_verosimilitud(y, mean(y), σ2_hat) + log_verosimilitud(x, mean(x), σ2_hat)  # Bajo H1

# Estadístico de prueba
LR = -2 * (logL0 - logL1)

# P-valor
p_value = 1 - cdf(Chisq(1), LR)
println("Razón de verosimilitudes: ", LR)
println("P-valor: ", p_value)


Razón de verosimilitudes: 0.8249739496052939
P-valor: 0.36372990193674193


# PROBLEMA 2 

### (a) Derivación de la distribución posterior condicional $\pi(\mu \mid \sigma^2, \mathbf{y})$

Dada la muestra $\mathbf{y} = (y_1, y_2, \dots, y_n)$, y suponiendo que $\sigma^2$ es conocida:

1. **Modelo**: Supongamos que los datos son independientes y normalmente distribuidos:
   $$
   y_i \mid \mu, \sigma^2 \sim \text{Normal}(\mu, \sigma^2), \quad i = 1, 2, \dots, n
   $$
   Es decir, la función de verosimilitud es:
   $$
   p(\mathbf{y} \mid \mu, \sigma^2) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(y_i - \mu)^2}{2\sigma^2}\right)
   $$
   Esto se puede escribir como:
   $$
   p(\mathbf{y} \mid \mu, \sigma^2) = \left(\frac{1}{\sqrt{2\pi\sigma^2}}\right)^n \exp\left(-\frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - \mu)^2\right)
   $$

2. **Prior de $\mu$**: El prior de $\mu$ es una distribución normal:
   $$
   \mu \sim \text{Normal}(\mu_0, \sigma_0^2)
   $$
   Entonces, la función de densidad del prior es:
   $$
   p(\mu) = \frac{1}{\sqrt{2\pi\sigma_0^2}} \exp\left(-\frac{(\mu - \mu_0)^2}{2\sigma_0^2}\right)
   $$

3. **Posterior de $\mu$**: La posterior de $\mu$ dado $\sigma^2$ y los datos es proporcional al producto de la verosimilitud y el prior:
   $$
   p(\mu \mid \sigma^2, \mathbf{y}) \propto p(\mathbf{y} \mid \mu, \sigma^2) p(\mu)
   $$
   Sustituyendo las expresiones de la verosimilitud y el prior:
   $$
   p(\mu \mid \sigma^2, \mathbf{y}) \propto \exp\left(-\frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - \mu)^2 - \frac{1}{2\sigma_0^2} (\mu - \mu_0)^2\right)
   $$

4. **Completando el cuadrado**: Simplificamos la expresión completando el cuadrado en $\mu$:
   $$
   p(\mu \mid \sigma^2, \mathbf{y}) \propto \exp\left(-\frac{1}{2}\left(\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}\right) \left(\mu - \mu_n\right)^2\right)
   $$
   donde:
   $$
   \mu_n = \frac{\frac{\mu_0}{\sigma_0^2} + \frac{n\bar{y}}{\sigma^2}}{\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}}, \quad \sigma_n^2 = \left(\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}\right)^{-1}
   $$
   Así, se obtiene que:
   $$
   \mu \mid \sigma^2, \mathbf{y} \sim \text{Normal}(\mu_n, \sigma_n^2)
   $$


In [78]:
using Distributions

# Parámetros a priori
μ0 = 0.0
σ0² = 1.0

# Datos simulados
y = [2.5, 3.1, 2.8, 3.6, 3.2]
n = length(y)
ȳ = mean(y)

# Dado un valor de σ²
σ² = 1.0  # Supón que es un valor conocido o inicial

# Parámetros de la distribución a posteriori de μ
μ_n = (σ0² * ȳ + σ² * μ0) / (n * σ0² + σ²)
σ_n² = (σ0² * σ²) / (n * σ0² + σ²)

# Distribución a posteriori condicional de μ dado σ² y y
posterior_μ = Normal(μ_n, sqrt(σ_n²))

# Print para la sección (a)
println("Distribución a posteriori condicional de μ | σ², y:")
println("μ ~ Normal($μ_n, $(sqrt(σ_n²)))")


Distribución a posteriori condicional de μ | σ², y:
μ ~ Normal(0.5066666666666667, 0.408248290463863)


### (b) Derivación de la distribución posterior condicional $\pi(\sigma^2 \mid \mu, \mathbf{y})$

Dada la muestra $\mathbf{y} = (y_1, y_2, \dots, y_n)$, y suponiendo que $\mu$ es conocida:

1. **Modelo**: Como antes, los datos siguen una distribución normal:
   $$
   y_i \mid \mu, \sigma^2 \sim \text{Normal}(\mu, \sigma^2)
   $$
   Entonces, la función de verosimilitud es:
   $$
   p(\mathbf{y} \mid \mu, \sigma^2) = \left(\frac{1}{\sqrt{2\pi\sigma^2}}\right)^n \exp\left(-\frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - \mu)^2\right)
   $$

2. **Prior de $\sigma^2$**: El prior de $\sigma^2$ es una distribución Inversa Gamma:
   $$
   \sigma^2 \sim \text{InverseGamma}(a, b)
   $$
   Entonces, la función de densidad del prior es:
   $$
   p(\sigma^2) \propto (\sigma^2)^{-(a+1)} \exp\left(-\frac{b}{\sigma^2}\right)
   $$

3. **Posterior de $\sigma^2$**: La posterior de $\sigma^2$ dado $\mu$ y los datos es proporcional al producto de la verosimilitud y el prior:
   $$
   p(\sigma^2 \mid \mu, \mathbf{y}) \propto p(\mathbf{y} \mid \mu, \sigma^2) p(\sigma^2)
   $$
   Sustituyendo las expresiones de la verosimilitud y el prior:
   $$
   p(\sigma^2 \mid \mu, \mathbf{y}) \propto (\sigma^2)^{-\frac{n}{2} - (a + 1)} \exp\left(-\frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - \mu)^2 - \frac{b}{\sigma^2}\right)
   $$

4. **Reconociendo la forma**: Esta expresión es de la forma de una distribución Inversa Gamma:
   $$
   p(\sigma^2 \mid \mu, \mathbf{y}) \sim \text{InverseGamma}\left(a_n, b_n\right)
   $$
   donde:
   $$
   a_n = a + \frac{n}{2}, \quad b_n = b + \frac{1}{2} \sum_{i=1}^{n} (y_i - \mu)^2
   $$


In [79]:
# Parámetros a priori
a = 2.0
b = 2.0

# Supón un valor conocido o inicial de μ
μ = 3.0  # Puedes usar el valor que sea relevante

# Parámetros de la distribución a posteriori de σ²
α_n = a + n/2
β_n = b + 0.5 * sum((y .- μ).^2)

# Distribución a posteriori condicional de σ² dado μ y y
posterior_σ² = InverseGamma(α_n, β_n)

# Print para la sección (b)
println("Distribución a posteriori condicional de σ² | μ, y:")
println("σ² ~ InverseGamma($α_n, $β_n)")


Distribución a posteriori condicional de σ² | μ, y:
σ² ~ InverseGamma(4.5, 2.35)


### (c) Implementación del Muestreo de Gibbs

El muestreo de Gibbs es una técnica de muestreo secuencial que se basa en muestrear de las distribuciones condicionales completas:

1. **Inicialización**: Asigne valores iniciales $\mu^{(0)}$ y $\sigma^{2(0)}$.
2. **Iteración**:
   - Muestree $\mu^{(t)} \sim \text{Normal}(\mu_n, \sigma_n^2)$ dado $\sigma^{2(t-1)}$.
   - Muestree $\sigma^{2(t)} \sim \text{InverseGamma}(a_n, b_n)$ dado $\mu^{(t)}$.

Se repite estos pasos para obtener una cadena de muestras $\{(\mu^{(t)}, \sigma^{2(t)})\}_{t=1}^{T}$ que convergerá a la distribución conjunta posterior.


In [80]:
iterations = 10000
burn_in = 2000
μ_samples = zeros(iterations)
σ²_samples = zeros(iterations)

# Valores iniciales
μ_samples[1] = mean(y)
σ²_samples[1] = var(y)

# Iteraciones del muestreo de Gibbs
for i in 2:iterations
    # Muestreo de μ dado σ², y
    σ² = σ²_samples[i-1]
    μ_n = (σ0² * ȳ + σ² * μ0) / (n * σ0² + σ²)
    σ_n² = (σ0² * σ²) / (n * σ0² + σ²)
    μ_samples[i] = rand(Normal(μ_n, sqrt(σ_n²)))
    
    # Muestreo de σ² dado μ, y
    α_n = a + n/2
    β_n = b + 0.5 * sum((y .- μ_samples[i]).^2)
    σ²_samples[i] = 1 / rand(Gamma(α_n, 1/β_n))
end

# Print para la sección (c)
println("Muestreo de Gibbs completado con $iterations iteraciones.")

# Mostrar algunas realizaciones de las muestras
println("\nAlgunas realizaciones de las muestras de μ y σ²:")
for i in 1:5  # Imprime las primeras 5 realizaciones como ejemplo
    println("Iteración $i: μ = $(μ_samples[i + burn_in]), σ² = $(σ²_samples[i + burn_in])")
end


Muestreo de Gibbs completado con 10000 iteraciones.

Algunas realizaciones de las muestras de μ y σ²:
Iteración 1: μ = 0.2512077461027267, σ² = 6.833180432768631
Iteración 2: μ = 0.17070272925518157, σ² = 19.502577936737215
Iteración 3: μ = 0.07177442053814564, σ² = 5.6618119008242545
Iteración 4: μ = -0.11176731920741267, σ² = 7.9465581503106275
Iteración 5: μ = 1.0004785080504304, σ² = 3.86140944668989





### (d) Inferencia a partir de la distribución posterior

Usando las muestras obtenidas del muestreo de Gibbs, se pueden calcular las inferencias puntuales (como la media posterior de $\mu$ y $\sigma^2$) y los intervalos de credibilidad (por ejemplo, los percentiles 2.5% y 97.5% para construir intervalos de credibilidad del 95%).

Cada uno de estos pasos se basa en derivaciones matemáticas de las propiedades de las distribuciones normales y de la distribución Inversa Gamma, junto con el uso de propiedades de completado del cuadrado y de muestreo secuencial en cadenas de Markov.


In [81]:
# Descartar el burn-in
μ_samples = μ_samples[burn_in+1:end]
σ²_samples = σ²_samples[burn_in+1:end]

# Calcular la media posterior y los intervalos creíbles
μ_mean = mean(μ_samples)
σ²_mean = mean(σ²_samples)
μ_cred_int = quantile(μ_samples, [0.025, 0.975])
σ²_cred_int = quantile(σ²_samples, [0.025, 0.975])

# Print para la sección (d)
println("Resultados de la inferencia:")
println("Media posterior de μ: $μ_mean")
println("Intervalo creíble del 95% para μ: $μ_cred_int")
println("Media posterior de σ²: $σ²_mean")
println("Intervalo creíble del 95% para σ²: $σ²_cred_int")


Resultados de la inferencia:
Media posterior de μ: 0.2933747548496111
Intervalo creíble del 95% para μ: [-1.2035338934446984, 1.6887523783577925]
Media posterior de σ²: 6.517111404570587
Intervalo creíble del 95% para σ²: [1.2532430193759119, 20.310649598027446]
